# Analysis 2

In [1]:
import argparse
import json
import pprint
import requests
import sys
import urllib
import csv
import time
import datetime
import pandas as pd
import glob
from pandas.io.json import json_normalize
from numpy import nan as NA

from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [2]:
CLIENT_ID = 'gBcvfFpabk3CnzmXKdYe3w'
CLIENT_SECRET = 'K9dIltpo1DH4hTrKy6c7YhxR2bwENElpzqTEaRz44QXxKnImd3cSOnYmhrucVHns'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'
SEARCH_LIMIT = 50

In [3]:
def obtaintoken(host, path):
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    token = response.json()['access_token']
    return token

token = obtaintoken(API_HOST, TOKEN_PATH)
token


'Z1C_ixTGccfADwGnwzG5yUE_tqYbJ3Vfj33NrA5Qi_LhXdj_A0idBPo8mP2dHU71s6HAkJuCJDxhH-BJ1WSDno-pyfmit-GNuVB8gPlu2kxjw-tgGZSO6VHnG4H_WHYx'

In [4]:
def request(host, path, token, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % token,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def search(token, term, location):
    

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, token, url_params=url_params)

def get_business(token, business_id):
    
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, token)

def query_api(term, location):
   
#     token = obtaintoken(API_HOST, TOKEN_PATH)
    response = search(token, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(token, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


In [ ]:
def findallresult():
    page = 0
    url_params = {
                'term': '',
                'location': 'Boston, MA',
                'limit': SEARCH_LIMIT,
                'open_at':unixtime,               # Unix Timestamp
                'offset':0
            }
    response1 = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
    with open('data/midnightsnack/page0.json', 'a') as outfile:
        json.dump(response1, outfile)

    while (page < (response1['total']/50)):
#    print 'The count is:', count
        page = page + 1
        url_params = {
                'term': '',
                'location': 'Boston, MA',
                'limit': SEARCH_LIMIT,
                'open_at':unixtime,               # Unix Timestamp
                'offset':50*page

            }

        response = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
        with open('data/midnightsnack/page'+ str(page) +'.json', 'a') as outfile:
            json.dump(response, outfile)